In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Target:

    Max epochs = 15
    Number of parameters < 10k
    Adding GAP layer

Result:

    1st run:
      Change: Added GAP layer with kernel size = 5
      Parameters : 8,634
      Train accuracy = 98.80%
      Test accuracy = 98.55%

    2nd run : 
      Change : GAP layer with kernel size = 7
               Added extra convolution layer before GAP
      Parameters : 8,634
      Train accuracy = 98.54%
      Test accuracy = 98.28%

    3rd run:
      Change : Added a layer after GAP
      Parameters : 9,674
      Train accuracy : 98.69%
      Test accuracy : 97.59%

    4th run:
      Change : Removed the second max pool layer, added more conv layers in its place, reduced channels to keep the parameters below 10k
      Parameters : 8,838 
      Train accuracy : 98.85%
      Test accuracy : 99.13%

Analysis:

    1st iteration - Changed the skeleton, removed 3 conv layers and added a GAP layer - Got max accuracy of 98.55%
    2nd iteration - Added a conv layer before GAP, GAP layer with kernel size 7 - got accuracy of 98.28%
    3rd iteration - Accuracy reduced
    4th iteration - Accuracy improved, no over fitting, the model can be further worked upon to increase the accuracy
    


In [0]:
# Original network with less parameters, so that training is also faster
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop1 = nn.Dropout(0.1)  
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0)         #input - 28 OUtput - 26 RF
        self.conv2 = nn.Conv2d(16, 10, 3, padding=0)        #input - 26 OUtput - 24
        self.pool1 = nn.MaxPool2d(2, 2)                     #input - 24 OUtput - 12
        self.bnm2d1 = nn.BatchNorm2d(10) 
        self.conv3 = nn.Conv2d(10, 10, 3, padding=0)        #input - 12 OUtput - 10
        self.conv4 = nn.Conv2d(10, 16, 3, padding=0)        #input - 10 OUtput - 8
        # self.pool2 = nn.MaxPool2d(2, 2)
        self.bnm2d2 = nn.BatchNorm2d(16) 
        self.conv5 = nn.Conv2d(16, 16, 3, padding=0)        #input - 8  OUtput - 6
        self.conv6 = nn.Conv2d(16, 16, 3, padding=1)        #input - 6  OUtput - 6
        # self.conv7 = nn.Conv2d(16, 16, 3, padding=1)      #input - 6  OUtput - 6
        self.gap = nn.AvgPool2d(kernel_size=6)
        self.conv8 = nn.Conv2d(16,10,1)                     

    def forward(self, x):
        x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
        x = self.bnm2d2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x)))))))
        x = self.conv6(F.relu(self.drop1(self.conv5(x))))
        # x = self.conv7(x)
        x = self.gap(x)
        x = self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)
        

In [63]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 10, 24, 24]           1,450
           Dropout-4           [-1, 10, 24, 24]               0
         MaxPool2d-5           [-1, 10, 12, 12]               0
       BatchNorm2d-6           [-1, 10, 12, 12]              20
            Conv2d-7           [-1, 10, 10, 10]             910
           Dropout-8           [-1, 10, 10, 10]               0
            Conv2d-9             [-1, 16, 8, 8]           1,456
          Dropout-10             [-1, 16, 8, 8]               0
      BatchNorm2d-11             [-1, 16, 8, 8]              32
           Conv2d-12             [-1, 16, 6, 6]           2,320
          Dropout-13             [-1, 16, 6, 6]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
train_acc = []
test_acc = []
train_losses = []
test_losses = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        y_pred = model(data)

        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
# def __init__(self):
#     super(Net, self).__init__()
#     self.drop1 = nn.Dropout(0.1)  
#     self.conv1 = nn.Conv2d(1, 16, 3, padding=1)     #input - 28 Output - 28 GRF - 
#     self.conv2 = nn.Conv2d(16, 16, 3, padding=1)    #input - 28 Output - 28 GRF - 
#     self.pool1 = nn.MaxPool2d(2, 2)                 #input - 28 Output - 14 GRF - 
#     self.bnm2d1 = nn.BatchNorm2d(16)                #input - 14 Output - 14 GRF - 
#     self.conv3 = nn.Conv2d(16, 16, 3, padding=1)    #input - 14 Output - 14 GRF - 
#     self.conv4 = nn.Conv2d(16, 16, 3, padding=1)    #input - 14 Output - 14 GRF - 
#     self.pool2 = nn.MaxPool2d(2, 2)                 #input - 14 Output - 7  GRF - 
#     self.bnm2d2 = nn.BatchNorm2d(16)                #input - 7  Output - 7  GRF - 
#     # self.conv5 = nn.Conv2d(16, 16, 3)
#     # self.conv6 = nn.Conv2d(16, 10, 3)
#     self.conv7 = nn.Conv2d(16, 10, 3)
#     self.gap = nn.AvgPool2d(kernel_size=5)

# def forward(self, x):
#     x = self.pool1(self.drop1(F.relu(self.bnm2d1(self.conv2(self.drop1(F.relu(self.bnm2d1(self.conv1(x)))))))))
#     x = self.pool2(self.drop1(F.relu(self.bnm2d2(self.conv4(self.drop1(F.relu(self.bnm2d1(self.conv3(x)))))))))
#     # x = F.relu(self.conv6(F.relu(self.conv5(x))))
#     # x = self.conv6(F.relu(self.conv5(x)))
#     x = self.conv7(x)
#     x = self.gap(x) 

#     x = x.view(-1, 10)
#     return F.log_softmax(x)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.09281978756189346 batch_id=468 Accuracy=85.89: 100%|██████████| 469/469 [00:12<00:00, 38.07it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.1459, Accuracy: 6023/10000 (60.23%)



loss=0.1645670086145401 batch_id=468 Accuracy=96.81: 100%|██████████| 469/469 [00:12<00:00, 38.38it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.0231, Accuracy: 6707/10000 (67.07%)



loss=0.10403452068567276 batch_id=468 Accuracy=97.58: 100%|██████████| 469/469 [00:12<00:00, 38.05it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.6488, Accuracy: 7695/10000 (76.95%)



loss=0.13909229636192322 batch_id=468 Accuracy=98.78: 100%|██████████| 469/469 [00:11<00:00, 39.77it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.5535, Accuracy: 8021/10000 (80.21%)



loss=0.006018439773470163 batch_id=468 Accuracy=98.75: 100%|██████████| 469/469 [00:11<00:00, 39.34it/s]



Test set: Average loss: 0.6484, Accuracy: 7743/10000 (77.43%)



In [0]:
# def __init__(self):
#     super(Net, self).__init__()
#     self.drop1 = nn.Dropout(0.1)  
#     self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
#     self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
#     self.pool1 = nn.MaxPool2d(2, 2)
#     self.bnm2d1 = nn.BatchNorm2d(32) 
#     self.conv3 = nn.Conv2d(32, 16, 3, padding=1)
#     self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
#     self.pool2 = nn.MaxPool2d(2, 2)
#     self.bnm2d2 = nn.BatchNorm2d(16) 
#     # self.conv5 = nn.Conv2d(16, 10, 3)
#     # self.conv6 = nn.Conv2d(16, 16, 3)
#     self.conv7 = nn.Conv2d(16, 10, 3)
#     self.gap = nn.AvgPool2d(kernel_size=5)

# def forward(self, x):
#     x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
#     x = self.bnm2d2(self.pool2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x))))))))
#     # x = F.relu(self.conv6(F.relu(self.conv5(x))))
#     x = self.conv7(x)
#     x = self.gap(x)
#     x = x.view(-1, 10)
#     return F.log_softmax(x)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.058040838688611984 batch_id=468 Accuracy=87.36: 100%|██████████| 469/469 [00:12<00:00, 39.05it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1526, Accuracy: 9578/10000 (95.78%)



loss=0.19096799194812775 batch_id=468 Accuracy=96.93: 100%|██████████| 469/469 [00:11<00:00, 39.41it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1057, Accuracy: 9704/10000 (97.04%)



loss=0.10852020978927612 batch_id=468 Accuracy=97.59: 100%|██████████| 469/469 [00:11<00:00, 39.39it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0424, Accuracy: 9867/10000 (98.67%)



loss=0.05140615627169609 batch_id=468 Accuracy=98.80: 100%|██████████| 469/469 [00:11<00:00, 40.64it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0492, Accuracy: 9848/10000 (98.48%)



loss=0.014406204223632812 batch_id=468 Accuracy=98.80: 100%|██████████| 469/469 [00:11<00:00, 41.08it/s]



Test set: Average loss: 0.0502, Accuracy: 9855/10000 (98.55%)



In [0]:
# def __init__(self):
#     super(Net, self).__init__()
#     self.drop1 = nn.Dropout(0.1)  
#     self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
#     self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
#     self.pool1 = nn.MaxPool2d(2, 2)
#     self.bnm2d1 = nn.BatchNorm2d(16) 
#     self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
#     self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
#     self.pool2 = nn.MaxPool2d(2, 2)
#     self.bnm2d2 = nn.BatchNorm2d(16) 
#     # self.conv5 = nn.Conv2d(16, 16, 3, padding=1)
#     # self.conv6 = nn.Conv2d(16, 16, 3, padding=1)
#     self.conv7 = nn.Conv2d(16, 10, 3, padding=1)
#     self.gap = nn.AvgPool2d(kernel_size=7)

# def forward(self, x):
#     x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
#     x = self.bnm2d2(self.pool2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x))))))))
#     # x = F.relu(self.conv6(F.relu(self.conv5(x))))
#     x = self.conv7(x)
#     x = self.gap(x)
#     x = x.view(-1, 10)
#     return F.log_softmax(x)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.12798148393630981 batch_id=468 Accuracy=82.24: 100%|██████████| 469/469 [00:12<00:00, 37.91it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.2285, Accuracy: 9399/10000 (93.99%)



loss=0.2033967226743698 batch_id=468 Accuracy=96.31: 100%|██████████| 469/469 [00:11<00:00, 39.20it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1459, Accuracy: 9580/10000 (95.80%)



loss=0.14860843122005463 batch_id=468 Accuracy=97.14: 100%|██████████| 469/469 [00:11<00:00, 39.67it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0783, Accuracy: 9758/10000 (97.58%)



loss=0.0911831185221672 batch_id=468 Accuracy=98.54: 100%|██████████| 469/469 [00:11<00:00, 39.64it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0594, Accuracy: 9828/10000 (98.28%)



loss=0.014485458843410015 batch_id=468 Accuracy=98.51: 100%|██████████| 469/469 [00:11<00:00, 39.19it/s]



Test set: Average loss: 0.0681, Accuracy: 9798/10000 (97.98%)



In [0]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.drop1 = nn.Dropout(0.1)  
#         self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
#         self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.bnm2d1 = nn.BatchNorm2d(16) 
#         self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
#         self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.bnm2d2 = nn.BatchNorm2d(16) 
#         # self.conv5 = nn.Conv2d(16, 16, 3, padding=1)
#         # self.conv6 = nn.Conv2d(16, 16, 3, padding=1)
#         self.conv7 = nn.Conv2d(16, 16, 3, padding=1)
#         self.gap = nn.AvgPool2d(kernel_size=7)
#         self.conv8 = nn.Conv2d(16,10,1)

#     def forward(self, x):
#         x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
#         x = self.bnm2d2(self.pool2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x))))))))
#         # x = F.relu(self.conv6(x))
#         x = self.conv7(x)
#         x = self.gap(x)
#         x = self.conv8(x)
#         x = x.view(-1, 10)
#         return F.log_softmax(x)
        
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.13571323454380035 batch_id=468 Accuracy=76.18: 100%|██████████| 469/469 [00:12<00:00, 37.70it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1716, Accuracy: 9587/10000 (95.87%)



loss=0.10811540484428406 batch_id=468 Accuracy=95.93: 100%|██████████| 469/469 [00:12<00:00, 37.97it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1503, Accuracy: 9610/10000 (96.10%)



loss=0.07129981368780136 batch_id=468 Accuracy=97.16: 100%|██████████| 469/469 [00:12<00:00, 38.54it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0589, Accuracy: 9827/10000 (98.27%)



loss=0.02711615525186062 batch_id=468 Accuracy=98.56: 100%|██████████| 469/469 [00:12<00:00, 38.55it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0791, Accuracy: 9759/10000 (97.59%)



loss=0.012098257429897785 batch_id=468 Accuracy=98.69: 100%|██████████| 469/469 [00:11<00:00, 39.85it/s]



Test set: Average loss: 0.0770, Accuracy: 9759/10000 (97.59%)



In [66]:
# def __init__(self):
#     super(Net, self).__init__()
#     self.drop1 = nn.Dropout(0.1)  
#     self.conv1 = nn.Conv2d(1, 16, 3, padding=0)         #input - 28 OUtput - 26 RF
#     self.conv2 = nn.Conv2d(16, 10, 3, padding=0)        #input - 26 OUtput - 24
#     self.pool1 = nn.MaxPool2d(2, 2)                     #input - 24 OUtput - 12
#     self.bnm2d1 = nn.BatchNorm2d(10) 
#     self.conv3 = nn.Conv2d(10, 10, 3, padding=0)        #input - 12 OUtput - 10
#     self.conv4 = nn.Conv2d(10, 16, 3, padding=0)        #input - 10 OUtput - 8
#     # self.pool2 = nn.MaxPool2d(2, 2)
#     self.bnm2d2 = nn.BatchNorm2d(16) 
#     self.conv5 = nn.Conv2d(16, 16, 3, padding=0)        #input - 8  OUtput - 6
#     self.conv6 = nn.Conv2d(16, 16, 3, padding=1)        #input - 6  OUtput - 6
#     # self.conv7 = nn.Conv2d(16, 16, 3, padding=1)      #input - 6  OUtput - 6
#     self.gap = nn.AvgPool2d(kernel_size=6)
#     self.conv8 = nn.Conv2d(16,10,1)                     

# def forward(self, x):
#     x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
#     x = self.bnm2d2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x)))))))
#     x = self.conv6(F.relu(self.drop1(self.conv5(x))))
#     # x = self.conv7(x)
#     x = self.gap(x)
#     x = self.conv8(x)
#     x = x.view(-1, 10)
#     return F.log_softmax(x)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.37925708293914795 batch_id=468 Accuracy=72.49: 100%|██████████| 469/469 [00:11<00:00, 40.47it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1197, Accuracy: 9657/10000 (96.57%)



loss=0.05871385335922241 batch_id=468 Accuracy=96.30: 100%|██████████| 469/469 [00:11<00:00, 41.01it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0621, Accuracy: 9813/10000 (98.13%)



loss=0.042483191937208176 batch_id=468 Accuracy=97.45: 100%|██████████| 469/469 [00:11<00:00, 40.96it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0260, Accuracy: 9919/10000 (99.19%)



loss=0.07877282053232193 batch_id=468 Accuracy=98.76: 100%|██████████| 469/469 [00:11<00:00, 41.33it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0260, Accuracy: 9912/10000 (99.12%)



loss=0.033292483538389206 batch_id=468 Accuracy=98.85: 100%|██████████| 469/469 [00:11<00:00, 41.56it/s]



Test set: Average loss: 0.0289, Accuracy: 9913/10000 (99.13%)

